# Replicating Bhatt&Lee Numerically

For conciseness, I import many of the functions which are used as 'helpers' here from Bhatt_Lee_tools.py

I also test all of these in TEST_Bhatt_Lee_tools.py

So if you maky any changes, they'll be tested automatically. 

In [1]:
from Bhatt_Lee_tools import *
import matplotlib.pyplot as plt 
from time import time

importing tools for replicating Bhatt and Lee
running checks on Bhatt_Lee_tools
all tests successful


If one bond in a cluster of 4 is very strong, the lowest 4 states can be represented as a singlet/triplet with energies matching our eigenvalues.

Given

$$
J/4 \qquad -3J/4
$$

the spacing we expect to see for the singlet/triplet is 

$$
J_{34} + (J_{13} -J_{23})(J_{24}-J_{14})/(2 J_{12}) + O(1/J_{12}^2)
$$

We can eliminate $J_{12}$ and write the whole cluster as two spins with coupling

$$
J'_{34} = J_{34} + (J_{13} - J_{23})(J_{24} - J_{14})/(2J_{12})
$$

Now we also need to modify couplings with other spins to obtain

$$
J'_{35} \qquad J'_{45}
$$

We do this by fitting the eigenvalues of the reduced ham $\sum_{i<j=1}^5 H_{ij}$ within the subspace consisting of the low lying singlet triplet set of (1,2,3,4) and the doublet on 5, to those of a 3 spin cluster (3', 4', 5)

We can do this by simply getting the lowest 8 energies of the hamiltonian for (1,2,3,4) and 5

this is the function: get_projected_energies(j12, j13, j14, j23, j24, j34, j51, j52, j53, j54)


## Fit to these eigenvalues

Lets use these 8 energies to obtain

$$
J'_{35} \qquad J'_{45}
$$

For the three spins (3, 4, 5) by matching eigenvalues

The eigenvalues of three spins connected in such a way can be found. Here I indicate $\times$ to indicate the multiplicity
$$
\lambda_3 = \frac{1}{4}(j12+j13+j23) \times 4
$$
$$
\lambda_2 = \frac{1}{4} \left(2 \sqrt{\text{j12}^2-\text{j23} (\text{j12}+\text{j13})-\text{j12} \text{j13}+\text{j13}^2+\text{j23}^2}-\text{j12}-\text{j13}-\text{j23}\right) \times 2
$$
$$
\lambda_1 = \frac{1}{4} \left(-2 \sqrt{\text{j12}^2-\text{j23} (\text{j12}+\text{j13})-\text{j12} \text{j13}+\text{j13}^2+\text{j23}^2}-\text{j12}-\text{j13}-\text{j23}\right) \times 2
$$

this makes sense here since we have $2\otimes 2\otimes 2 = 4\oplus 2\oplus 2$

We'll first identify the levels (identify_levels(eigenvalues))

Then we'll fit values to these eigenvalues given $j34$ (fit_to_values(eigenvalues, j34)

## We can do this analytically. 

Suppose we identify three levels: $\rho_1$, $\rho_2$, $\rho_3$, and we are given $j23 = \gamma$

Then we can matche the following:
$$
\alpha \equiv \rho_2-\rho_1 = \lambda_2 - \lambda_1 = \sqrt{\text{j12}^2-\text{j12} (\text{j13}+\text{j23})+\text{j13}^2-\text{j13} \text{j23}+\text{j23}^2}
$$

$$
\beta \equiv \rho_3-\rho_1 =  \lambda_3 - \lambda_1 =\frac{1}{2} \left(\sqrt{\text{j12}^2-\text{j23} (\text{j12}+\text{j13})-\text{j12} \text{j13}+\text{j13}^2+\text{j23}^2}+\text{j12}+\text{j13}+\text{j23}\right)
$$
Now I'll define 
$$
\alpha = \sqrt{\text{j12}^2-\text{j23} (\text{j12}+\text{j13})-\text{j12} \text{j13}+\text{j13}^2+\text{j23}^2}
$$
so that
$$
\beta = \frac{1}{2} \left(\alpha+\text{j12}+\text{j13}+\gamma\right)
$$

also we have

$$
\delta \equiv \rho_3-\rho_2 =  \lambda_3 - \lambda_2 = \frac{1}{2} \left(-\sqrt{\text{j12}^2-\text{j23} (\text{j12}+\text{j13})-\text{j12} \text{j13}+\text{j13}^2+\text{j23}^2}+\text{j12}+\text{j13}+\text{j23}\right)
$$

$$
\delta = \frac{1}{2} \left(-\alpha+\text{j12}+\text{j13}+\gamma\right)
$$

So 
$$
\beta + \delta = j12 + j13 + \gamma \implies \beta + \delta - \gamma = j12 + j13
$$
$$
j12 = \beta + \delta - \gamma - j13
$$

we can plug this into $\alpha$ to get that 

$$
\alpha = \sqrt{
\beta ^2-3 \beta  \gamma +2 \beta  \delta +3 \gamma ^2-3 \gamma  \delta +\delta ^2+3 \text{j13}^2-3 \text{j13} (\beta -\gamma +\delta )
}
$$

Then we have that
$$
\beta - \delta = \alpha \implies (\beta-\delta)^2 = \beta ^2-3 \beta  \gamma +2 \beta  \delta +3 \gamma ^2-3 \gamma  \delta +\delta ^2+3 \text{j13}^2-3 \text{j13} (\beta -\gamma +\delta )
$$
Solve for j13
$$
\text{j13} = \frac{1}{6} \left(3 \beta -3 \gamma +3 \delta\pm \sqrt{3} \sqrt{3 \beta ^2+6 \beta  \gamma -10 \beta  \delta -9 \gamma ^2+6 \gamma  \delta +3 \delta ^2} \right)
$$
and so 
$$
\text{j12} = \frac{1}{6} \left(3 \beta -3 \gamma +3 \delta\mp \sqrt{3} \sqrt{3 \beta ^2+6 \beta  \gamma -10 \beta  \delta -9 \gamma ^2+6 \gamma  \delta +3 \delta ^2} \right)
$$

## The functions:

**get_energies(j12, j13, j14, j23, j24, j34)**: calculate numerically the lowest 4 eigenvalues of the spin cluster's hamiltonian. This is mainly for confirmation of what we expect to see.

returns: lowest 4 eigenvalues of the (1,2,3,4) hamiltonian

**get_projected_energies(j12, j13, j14, j23, j24, j34, j51, j52, j53, j54)**: numercially calculate the lowest 8 eigenvalues. These should be able to be mapped onto 3 spins later. 

returns: lowest 8 eigenvalues of the full (1,2,3,4) + 5 hamiltonian

**identify_levels(eigenvalues)**: Identify the three corresponding clusters, given 8 eigenvalues. We expect to have a grouping of 4 eigenvalues, and two groupings of 2 eigenvalues. Give the means of each of these three clusters

returns: [mean of 4-grouping, mean of 2-grouping, mean of 2-grouping]

**fit_to_values(eigenvalues, j34)**: Now we combine the above functions here. Given 8 eigenvalues, we identify the correspoinding 3 clusters. Then we find the effective j35, j45, which give us those

returns: [effective j35, effective j45]


In [2]:
"""
These functions are used below
renormalize_coupling takes coupling strengths between 1,2,3,4 and a fifth point 5

returns the effective couplings j35 and j45 by fitting
"""

def renormalize_couplings(coupling_strengths):
    """
    coupling_stengths is a list/tuple/array of the couplings of the cluster and th fifth point
    in the order:
    j12, j13, j14, j23, j24, j34, j51, j52, j53, j54
    
    return new couplings between p3 and p5 ,and p4 and p5
    """
    j12, j13, j14, j23, j24, j34, j51, j52, j53, j54 = coupling_strengths
    
    eigenvalues = get_projected_energies(j12, j13, j14, j23, j24, j34, j51, j52, j53, j54)
    
    #FOR DEBUGGING
    #we need to make sure that the levels we identify are correct
    
    print('eigenvalues:', eigenvalues)
    v1, v2, v3 = split_up_eigenvalues(eigenvalues)
    print(v1)
    print(v2)
    print(v3)
    print("j12, j13, j14, j23, j24, j34, j51, j52, j53, j54")
    print(j12, j13, j14, j23, j24, j34, j51, j52, j53, j54)
    print(" ")
    
    
    newj34 = j34 + (j13 - j23)*(j24 - j14)/(2*j12)
    
    j35, j45 = fit_to_values(eigenvalues, newj34)
    
    return j35, j45

# Making the setup

Now we spread particles throughout space and enact the rg procedure

$$
J(r) = J_0 \exp(-2 r/a)
$$
with $n a^D \ll 1$, we place 10000 randomly in a 2d plane. 

$$
10000\times a^2 \ll 1 \implies a \ll 1/100
$$

let's take $a = (1/100)^2$

We enforce a threshold, $J_c$, so that total interaction strengths less than the thresshold are removed. We should set $J_c$ so that there are initially 
15-30 couplings. 

We'll also set $J_0 =1$ without loss of generality


# space vs time tradeoff

In order to deal with this large array, we make use of space. 

This means that we will make helper arrays which store information for us

**points**: this is an array of all the points of shape (10000, 2)

**point_dict**: this is a dictionary which tells us the connections to a point, and their coupling strength in order from strongest to weakest. In order to use this on a point, we need to convert the point to a tuple e.g. np.array([.5, .6]) -> (.5, .6) 

(since dictionary keys must be immutable, and tuples are immutable).

E.g.: point_dict[(.1, .45)] = [((.105, .5), .7), ((.2, .1), .6),((.7, .9), .2)]

This point is connected to 3 points, with couplings .7, .6, and .2
 
$$ $$

Also, in order to find these couplings in the first place, we use a discretization of space.


**space_dict** is a dictionary which maps regions in space to the points in them. But it is only used in the definition of make_arrangement

by using memory (making dictionaries), we convert the process of creating an arrangement of these points from one that is $O(N^2)$ to one that is $\sim O(N)$

In [3]:
"""
Define a function to initialize a setup
This is included here in case modification is needed
"""
def make_arrangement(Jc=.01, a=(1/250), N = 10000):
    points = np.random.rand(N, 2)
    
    #dictionary listing points close enough to count
    #as well as their coupling strength: (point, coupling)
    point_dict = dict()
    
    #space dictionary:
    #because we have so many points, we'll make use of space 
    #and speed things up
    
    #dx should be the max distance that the cutoff allows
    dx = -a*np.log(Jc)
    
    #a dictionary to hold the points in a region
    #then to find connections, we'll just look at the neighboring boxes: (x \pm dx, y \pm dx)
    #To avoid machine precision errors, we'll round to 8 decimal places here
    
    num_slices = len(np.arange(0, 1, dx))
    space_dict = {(x, y): [] for x in range(num_slices) for y in range(num_slices)}
    
    def map_to_point(x, y):
        """
        given a point in our region, what is the corresponding block
        in space_dict
        """
        return int(x/(dx)), int(y/(dx))
    
    for p in points:
        coresponding_i, corresponding_j = map_to_point(p[0], p[1])
        space_dict[(coresponding_i, corresponding_j)].append(p)
    
    point_dict = {(p[0], p[1]) : [] for p in points}
    
    for i, p in enumerate(points):
        tuple_p = (p[0], p[1])
        #point_dict[tuple_p] = []
        
        """
        Construct a list of points to check
        """
        close_points = []
        x, y = map_to_point(p[0], p[1])
        steps = [(x,y), (x, y+1), (x+1, y), (x, y-1), (x-1, y),
                 (x+1, y+1), (x+1, y-1), (x-1, y+1), (x-1, y-1)]
        
        #periodic bcs
        steps = map(lambda x: (x[0] % num_slices, x[1] % num_slices), steps)
        steps = list(set(steps))
        
        for step in steps:
            if step in space_dict:
                close_points+=space_dict[step]
        
        #take note of points we've already included
        if point_dict[tuple_p]:
            prev_points, _ = zip(*point_dict[tuple_p]) 
        else:
            prev_points = []
        
        for j, p2 in enumerate(close_points):
            #check that it hasn't already been included
            if tuple(p2) not in prev_points:
                #check that its above the threshold
                #since we have periodic bc's we need to consider 
                #shifted p2's 

                #are we close enough to care?
                #are we within dx of the boundary
                if min(p2[0], 1-p2[0])<=dx or min(p2[1], 1-p2[1])<=dx:
                    #we're close enough to the boundary for periodic bc's to matter
                    radii = []
                    for xstep in [-1, 0, 1]:
                        for ystep in [-1, 0, 1]:
                            shifted_point = (p2[0]+xstep, p2[1]+ystep)
                            radii.append(LA.norm(p-shifted_point))

                    r = np.min(radii)
                else:
                    r = LA.norm((p[0]-p2[0], p[1]-p2[1]))

                coupling = np.exp(-2*r/a)

                if coupling>=Jc and r!=0:
                    #add it to dictionary
                    point_dict[tuple_p].append((tuple(p2), coupling))
                    point_dict[tuple(p2)].append((tuple_p, coupling))
        
        #now sort by coupling strength
        #with the strongest couplings coming first
        point_dict[tuple_p].sort(key = lambda x: -x[-1])
    

    #now return the points and the dictionary 
    return points, point_dict

## test make_arrangement

In [4]:
"""
Now we test this on a small enough N 
So that we can compute everything manually, and make sure our
faster version agrees
"""
test_Jc = .01
test_a = 1/110
test_N=100

"""
We'll run this a few times since randomness is involved to make sure 
it really does work
"""
for rep in range(60):
    if rep%5==0:
        print('test ',rep)
    test_points, test_point_dict = make_arrangement(Jc=test_Jc, a=test_a, N = test_N)
    
    #cornfim that test_point_dict matches what it should be
    confirmation_point_dict = {(p[0], p[1]) : [] for p in test_points}

    for i in range(test_points.shape[0]):
        p = (test_points[i][0], test_points[i][1])
        
        if confirmation_point_dict[p]:
            prev_points, _ = zip(*confirmation_point_dict[p])
        else:
            prev_points = []
        
        for j in range(test_points.shape[0]):
            if j != i and (test_points[j][0], test_points[j][1]) not in prev_points:
                p2 = (test_points[j][0], test_points[j][1])
                #compute the coupling
                radii = []
                for xstep in [-1, 0, 1]:
                    for ystep in [-1, 0, 1]:
                        shifted_point = (p2[0]+xstep, p2[1]+ystep)
                        radii.append(LA.norm((p[0]-shifted_point[0], p[1]-shifted_point[1])))

                r = np.min(radii)
                coupling = np.exp(-2*r/(test_a))
                
                #is it greater than test_Jc
                if coupling>=test_Jc:
                    confirmation_point_dict[p].append((p2, coupling))
                    confirmation_point_dict[p2].append((p, coupling))
        confirmation_point_dict[p].sort(key = lambda x: -x[-1])


        if test_point_dict[p] != confirmation_point_dict[p]:
            print(p, test_point_dict[p])
            print(p, confirmation_point_dict[p])
            print('----')

    assert(confirmation_point_dict == test_point_dict)

print('all correct here')

test  0


KeyboardInterrupt: 

## Make sure that we have parameters s.t. each point has 15-30 couplings

In [ ]:
points, point_dict = make_arrangement(Jc=5*10**-9, a=0.0028, N = 10000)

histogram = [0 for _ in range(10)]
for p in points:
    num_couplings = len(point_dict[(p[0], p[1])])
    index=num_couplings//5
    histogram[index]+=1

xs = [str(i*5)+"-"+str((i+1)*5) for i in range(10)]

plt.bar(xs, histogram)
plt.title('distribution of number of couplings')
plt.show()


Jc=5 * 10**-9, a=0.0028, N = 10000 seems to work okay

## functions for clustering

**max_coupling(points, point_dict)** This function returns the maximum coupling present, and the two points with that coupling

returns: Jmax, [p1, p2]

**check_cluster(cluster, point_dict)** This confirms that given a cluster (of the form (p1, p2, p3, p4)) is a cluster (i.e. all points are connected to all other points in the cluster)

returns: Bool (true/False)

**get_cluster_couplings(cluster, point_dict)** given a cluster, what are the couplings? 

returns: J01, J02, J03, J12, J13, J23

**max_cluster(points, point_dict)** Now we use these functions together in max_cluster. This returns the cluster which is most strongly coupled -- that is, it has the strongest coupling in it, $J_max$, and of the clusters containing $J_{max}$, its smallest coupling is the largest. 

returns: cluster, cluster strengths 

cluster strengths are of the form: J01, J02, J03, J12, J13, J23

In [ ]:
def max_cluster(points, point_dict):
    """
    Find a spatially tight cluster of points
    """
    #first identify the strongest coupling
    max_J, points = max_coupling(points, point_dict)
    
    """
    get connecting points
    """
    
    connections0 = point_dict[points[0]]
    connections1 = point_dict[points[1]]
    
    """
    now we look for connecting points between these sets of points
    """
    points0, _ = zip(*connections0)
    points1, _ = zip(*connections1)
    
    possible_clusters = []
    
    for p0 in points0:
        for p1 in points1:
            #check if its a possible cluster (all points are connected to one another)
            if check_cluster((points[0], points[1], p0, p1), point_dict):
                possible_clusters.append((points[0], points[1], p0, p1))
    
    """
    what if we have found zero clusters to use
    Let's raise an assertion error if this occurs
    We can deal with this as it arises
    """
    assert(len(possible_clusters)>0)
    
    """
    of these possible clusters, lets choose the one with the strongest
    couplings
    
    By strongest we mean that 
    min(np.abs(current_coupling)) 
    is maximized
    """
    min_coupling = []
    coupling_strengths = []
    
    for cluster in possible_clusters:
        current_coupling = get_cluster_couplings(cluster, point_dict)
        min_coupling.append(min(np.abs(current_coupling)))
        coupling_strengths.append(current_coupling)
    
    max_index = np.argmax(min_coupling)
    
    favored_cluster = possible_clusters[max_index]
    
    return favored_cluster, coupling_strengths[max_index]
    
        

In [ ]:
import cProfile
cProfile.run('make_arrangement(Jc=5*10**-9, a=0.0028, N = 10000)')

## test max_cluster

In [ ]:
test_Jc = .01
test_a = 1/10
test_N = 1000

test_points, test_point_dict = make_arrangement(Jc=test_Jc, a=test_a, N = test_N)
"""
make one connection with a huge coupling
between p1 and p2
"""
Jmax = 100

p1 = tuple(test_points[0])
points1, strengths1 = zip(*test_point_dict[p1])

p2 = points1[0]
points2, strengths2 = zip(*test_point_dict[p2])

strengths1, strengths2 = list(strengths1), list(strengths2)
#now reset the coupling
strengths1[points1.index(p2)]= Jmax
strengths2[points2.index(p1)]= Jmax

test_point_dict[p1] = sorted(zip(points1, strengths1), key = lambda x: -x[-1])
test_point_dict[p2] = sorted(zip(points2, strengths2), key = lambda x: -x[-1])

cluster, strengths = max_cluster(test_points, test_point_dict)

assert(max(strengths)==Jmax)
assert(p1 in cluster and p2 in cluster)
print('looks good')

## Tools for RG

Now we come to the tools for actually performing the rg

**remove_point(points_to_remove, points, point_dict)**: This returns a new *points* and *point_dict* with the points in *points_to_remove* taken out. 

returns: points, point_dict

**update_coupling(p1, p2, Jnew, point_dict)**: This returns a new *point_dict* with the connections between p1 and p2 updated

returns: point_dict





In [ ]:
def reduce_strongest_cluster(points, point_dict):
    """
    given points, and a point dictionary:
    
    1) identify the strongest coupled cluster and its strengths
    2) identify all points, p5, connected to the cluster
    3) update all j35, j45 using 'renormalize couplings'
    4) remove p1, p2
    5) update j34. This should be done last, since the renormalized 
       j35 and j34 must be calculated using the point dict giving 
       all original couplings for the cluster
    
    """
    #identify the largest coupling
    cluster, strengths = max_cluster(points, point_dict)
    
    p1, p2, p3, p4 = cluster
    j12, j13, j14, j23, j24, j34 = strengths
    
    
    #this should be j12
    assert(j12==max(strengths))
    
    
    """
    now go through all connections
    add them to a record of p5s
    and renormalize their couplings
    """
    
    p5_dict = dict()
    
    for i, p in enumerate(cluster):
        p5s, strengths_to_p5 = zip(*point_dict[p1])
        
        for j, p5 in enumerate(p5s):
            """
            Note p5s must not be in the cluster already
            """
            if p5 not in cluster:
                if p5 not in p5_dict:
                    #set couplings to 1,2,3,4 to be zero at first
                    p5_dict[p5] = [0,0,0,0]
                else:
                    #set the ith coupling here to be ji5
                    p5_dict[p5][i] = strengths_to_p5[j]

    for p5 in p5_dict:
        #now add the strengths of the (1,2,3,4) cluster to the front
        #since they are necessary for getting the renormalized coupling
        p5_dict[p5] = list(strengths)+p5_dict[p5]
        
        #print(p5_dict[p5])
    
    assert(set(p5_dict.keys()).intersection(set(cluster)) == set())
    """
    Now find the updates that each coupling needs
    """
    for p5 in p5_dict:
        new_j35, new_j45 = renormalize_couplings(p5_dict[p5])
        
        #print('p5: ',p5)
        #print('new couplings', new_j35, new_j45)
        
        #and apply these updates to obtain new point_dict
        point_dict = update_coupling(p3, p5, new_j35, point_dict)
        point_dict = update_coupling(p4, p5, new_j45, point_dict)
    
    """
    remove p1, p2
    """
    points, point_dict = remove_point((p1, p2), points, point_dict)
    
    
    """
    update the j34 coupling
    """
    lowest_4 = get_energies(j12, j13, j14, j23, j24, j34)
    #print('points in cluster', p1, p2, p3, p4)
    #print("j12, j13, j14, j23, j24, j34", j12, j13, j14, j23, j24, j34)
    #print('j12 :', j12)
    #print(lowest_4)
    #print('predicted gap: ', j34 + (j13 - j23)*(j24 - j14)/(2*j12))
    #print('actul gap      ', lowest_4[1] - lowest_4[0])
    
    newj34 = j34 + (j13 - j23)*(j24 - j14)/(2*j12)
    
    if newj34<0:
        print('newj34 LOWER THAN ZERO -- this presents an issue', newj34)
        #print("setting to zero \n")
        #newj34=0
        print("new j34 ", newj34)
        print(" ")
        
    assert(len(set(cluster))==len(cluster))
    #assert(newj34>=0)
    
    point_dict = update_coupling(p3, p4, newj34, point_dict)
    
    """
    return the new setup
    """
    return points, point_dict 

# test reduce_strongest_cluster

In [ ]:
test_Jc = .01
test_a = 1/10
test_N = 1000

test_points, test_point_dict = make_arrangement(Jc=test_Jc, a=test_a, N = test_N)
"""
make one connection with a huge coupling
between p1 and p2
"""
Jmax = 100

p1 = tuple(test_points[0])
points1, strengths1 = zip(*test_point_dict[p1])

p2 = points1[0]
points2, strengths2 = zip(*test_point_dict[p2])

strengths1, strengths2 = list(strengths1), list(strengths2)
#now reset the coupling
strengths1[points1.index(p2)]= Jmax
strengths2[points2.index(p1)]= Jmax

test_point_dict[p1] = sorted(zip(points1, strengths1), key = lambda x: -x[-1])
test_point_dict[p2] = sorted(zip(points2, strengths2), key = lambda x: -x[-1])


"""
Apply cluster reduction
"""

test_points, test_point_dict = reduce_strongest_cluster(test_points, test_point_dict)

"""
lets check that p1, p2 are removed from everything
"""

for i in range(test_points.shape[0]):
    point = tuple(test_points[i])
    assert(point!=p1)
    assert(point!=p2)

assert(p1 not in test_point_dict)
assert(p2 not in test_point_dict)

assert(test_points.shape[0] == test_N-2)

"""
For now I'm happy with this
I can test the couplings later
"""
print('appears fine')

# Setting up the system and running RG

of course to set up an arrangement we run 

make_arrangement(Jc=.005, a=(1/100), N = 10000)

Then to apply the rg, we apply reduce_strongest_cluster. I'll combine that into a function below called

**apply_rg(points, point_dict)**

This function allows us to more neatly work with the procedure. 

We can add analyses in the body and then iteratively apply the function to run the rg flow.

In [ ]:
def apply_rg(points, point_dict):
    """
    Apply rg once
    
    this is packaged so that if we want we can add in functions to track the distributions of the 
    coupling strength without modifying the functions which are actually doing the work.
    """
    assert(len(points)==len(point_dict))
    new_points, new_point_dict = reduce_strongest_cluster(points, point_dict)
    
    
    couplings = []
    for p in new_points:
        _, strengths = zip(*new_point_dict[(p[0], p[1])])
        couplings+= strengths
    
    #remove zeros
    couplings = list(filter(lambda x: x!= 0, couplings))
    
    mean_of_log = np.mean(np.log(np.abs(np.array(couplings))))
    var_of_log = np.var(np.log(np.abs(np.array(couplings))))
    J_max, _ = max_coupling(new_points, new_point_dict)
    
    dist_log_width = np.std(np.log(np.abs(np.array(couplings))/J_max))
    
    return new_points, new_point_dict, mean_of_log, var_of_log, J_max, dist_log_width

In [ ]:
def iterate_rg(N):
    """
    Apply rg iteratively N times and return various statistics of the rg flow
    """
    #points, point_dict = make_arrangement(Jc=.005, a=(1/100), N = 10000)

    points, point_dict = make_arrangement(Jc=5*10**-9, a=0.0028, N = 10000)
    
    mean_of_logs, var_of_logs, max_couplings, log_width = [], [], [], []
    
    for i in range(N):
        
        if N>100:
            """
            We'll monitor the time this takes if N is large enough
            Here I take 'large enough' to be 100
            """
            if i==0:
                initial_time = time()
            """
            Here I print the remaining minutes left after every 25 iterations
            """
            if i>0 and i%25==0:
                time_elapsed = time()-initial_time
                mean_time_per_iteration = time_elapsed/i
                total_expected_time = mean_time_per_iteration*N
                total_time_remaining = total_expected_time - time_elapsed
                print("iteration: ", i)
                print("minutes remaining: ", round(total_time_remaining/60, 2))
                print("---------------------------")
                
        
        points, point_dict, mean_of_log, var_of_log, J_max, dist_log_width = apply_rg(points, point_dict)
        
        max_couplings.append(J_max)
        mean_of_logs.append(mean_of_log)
        var_of_logs.append(var_of_log)
        log_width.append(dist_log_width)
    
    return mean_of_logs, var_of_logs, max_couplings, log_width
    

In [ ]:
out = iterate_rg(N = 3000)

In [ ]:
mean_of_logs, var_of_logs, max_couplings, log_width = out

In [ ]:
plt.plot(log_width)

In [ ]:
plt.plot(max_couplings/max_couplings[0])
plt.yscale('log')
plt.show()

In [ ]:
plt.plot(np.exp(mean_of_logs)/np.exp(mean_of_logs)[0])
#plt.yscale('log')
plt.show()

In [ ]:
plt.plot(np.exp(mean_of_logs)/max_couplings)
#plt.yscale('log')
plt.show()

In [ ]:
plt.plot(np.log(max_couplings))
plt.plot(mean_of_logs)
plt.show()

In [ ]:

plt.plot(np.exp(mean_of_logs))
plt.show()

In [ ]:
plt.plot(max_couplings/max_couplings[0], np.sqrt(var_of_logs)/np.sqrt(var_of_logs[0]))
plt.show()